<a href="https://colab.research.google.com/github/llindish/Complete-Python-3-Bootcamp/blob/master/WTD_model/GEE_code/Forsinard_S1_mosaics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

In [2]:
#Authentification needed only once, first time you run the colab code
from google.colab import auth
auth.authenticate_user()

In [3]:
cloud_project = 'geographic-information-systems'
ee.Initialize(project=cloud_project)
geemap.ee_initialize(project=cloud_project)

In [4]:
#monthly Senitnel-1 iamgery for Forsinard

###########################
#ADJUST FOLLOWING 4 variables

# !Specify the asset (.shp file of the loggers stored on GEE as asset)
asset_id = 'users/llindish/Forsinard_all_30m'

#burst shapefile
burst = 'projects/ee-llindish/assets/030_062635_IW2_Forsinard_burst_frame'

# !Specify the year.
year = 2018

# !Specify the name of your bucket(where would you like to export) and the specific location in the bucket
bucket = 'soilmoisture-jbdsag'
path = 'GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018'

# !Specify the boundary box (the image area is based on these coordinates!)
coords = [[-4.079731012164764,58.271659286472214],
          [-3.6210518129460145,58.271659286472214],
          [-3.6210518129460145,58.46037179423765],
          [-4.079731012164764,58.46037179423765],
          [-4.079731012164764,58.271659286472214]]

################################################

# Change assest into a feature collection
geometry = ee.FeatureCollection(asset_id)

# Create the polygon
aoi = ee.Geometry.Polygon(coords)

#specify orbit direction and path
path_dir = 'ASCENDING' #or 'DESCENDING'
path_orb = 30 #30 for Forsinar

# Load Sentinel-1 ImageCollection.
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.calendarRange(year, year, 'year')) \
    .filter(ee.Filter.eq('orbitProperties_pass', path_dir)) \
    .filter(ee.Filter.eq('relativeOrbitNumber_start', path_orb ))

#check number of images
print("Number of images in the collection:", sentinel1.size().getInfo())

# Add an additional filter to include only images that fully cover the burst shapefile
sentinel1 = sentinel1.filterBounds(ee.FeatureCollection(burst))
print("Number of images in the filtered collection:", sentinel1.size().getInfo())


# Function to calculate monthly mean/median composite.
#TRY median/min or max??

def monthlyComposite(month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    monthly_images = sentinel1.filterDate(start_date, end_date)
    mean_image = monthly_images.mean()
    return mean_image.rename(['VV', 'VH', 'angle']).toFloat()

# Generate monthly composites for a year.
months = ee.List.sequence(1, 12)
monthlyComposites = ee.ImageCollection.fromImages(months.map(monthlyComposite))

def filter_contains(img):
    bounds = img.geometry()
    return ee.Algorithms.If(bounds.contains(aoi), img)

filtered_collection = monthlyComposites.map(filter_contains)
filtered_collection = filtered_collection.filter(ee.Filter.neq('system:index', None))

def export_image_collection_to_cloud_storage(img_coll, bucket, folder):
    """Exports an image collection to a specific folder in a Google Cloud Storage bucket."""
    def export_image(img):
        """Inner function to export a single image."""
        # Get the image's date and format it as a string in the format 'YYYY-MM'.
        date = year
        # Construct the file name prefix.
        name = int(img.get('system:index').getInfo())
        name = name + 1
        #SPECIFY THE FILE NAME
        file_name_prefix = f"{folder}/Forsinard_S1_monthly_{date}_{name}"
        print(f"Exporting {file_name_prefix}")
        export_options = {
            'image': img,
            'bucket': bucket,
            'fileNamePrefix': file_name_prefix,
            'region': aoi,
            'scale': 10,
            'maxPixels': 1e10,
            'crs': 'EPSG:27700'
        }
        task = ee.batch.Export.image.toCloudStorage(**export_options)
        task.start()

    img_coll_list = img_coll.toList(img_coll.size())
    for i in range(img_coll_list.size().getInfo()):
        img = ee.Image(img_coll_list.get(i))
        export_image(img)

# Initialize ImageCollection
image_collection = filtered_collection
# Specify bucket name
bucket_name = bucket
# Specify folder path
folder_path = path

# Call the function
export_image_collection_to_cloud_storage(image_collection, bucket_name, folder_path)


Number of images in the collection: 119
Number of images in the filtered collection: 119
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_1
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_2
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_3
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_4
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_5
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_6
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_7
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_8
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_9
Exporting GEE_exports_LT/S1/Forsinard/Brit_grid_27700/2018/Forsinard_S1_monthly_2018_10
Exporting GEE_exports_LT/S1/Forsinard/Br

In [ ]:
#Visualisatioz on GEEmap

map = geemap.Map()
url = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}"
map.add_tile_layer(url, name="Google Satellite", attribution="Google")

# Centre the map view over the ROI
map.centerObject(geometry, 13)

# Select the first image from the collection.
firstImage = ee.Image(monthlyComposites.first())

# Select the 'VH' band.
vhBand = firstImage.select('VH').clip(aoi)

# Select the 'angle' band.
angle= firstImage.select('angle').clip(aoi)

# Add the 'VH' band of the first image to the map.
map.addLayer(vhBand, {'min': -25, 'max': 0}, 'January VH Image')

# Add the 'angle' band of the first image to the map.
#map.addLayer(angle, {'min': 40, 'max': 42}, 'angle')

map.addLayer(geometry)
map

Map(center=[58.394536127699844, -3.8406067140690543], controls=(WidgetControl(options=['position', 'transparen…